In [16]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchaudio
from python_speech_features import mfcc
import textgrids
import os

In [23]:
directory = '../audioData/VAD_Kaggle/Data/Audio'
annot_dir = '../audioData/VAD_Kaggle/Data/Annotation'

aud_files = []
annotation_files = []

for subdir, dirs, files in os.walk(directory):
    for file in files:
        fpath = subdir+os.sep+file
        aud_files.append(fpath)
        #print(fpath)

for subdir, dirs, files in os.walk(annot_dir):
    for file in files:
        fpath = subdir+os.sep+file
        annotation_files.append(fpath)
        #print(fpath)

print(len(aud_files), len(annotation_files))

719 719


In [25]:
dataset_csv = pd.DataFrame(columns=['audio_files', 'annotation_files'])
dataset_csv['audio_files'] = aud_files
dataset_csv['annotation_files'] = annotation_files

dataset_csv

,audio_files,annotation_files
0,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,../audioData/VAD_Kaggle/Data/Annotation/Female...
1,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,../audioData/VAD_Kaggle/Data/Annotation/Female...
2,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,../audioData/VAD_Kaggle/Data/Annotation/Female...
3,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,../audioData/VAD_Kaggle/Data/Annotation/Female...
4,../audioData/VAD_Kaggle/Data/Audio/Female/PTDB...,../audioData/VAD_Kaggle/Data/Annotation/Female...
...,...,...
714,../audioData/VAD_Kaggle/Data/Audio/Noizeus/Tra...,../audioData/VAD_Kaggle/Data/Annotation/Noizeu...
715,../audioData/VAD_Kaggle/Data/Audio/Noizeus/Tra...,../audioData/VAD_Kaggle/Data/Annotation/Noizeu...
716,../audioData/VAD_Kaggle/Data/Audio/Noizeus/Tra...,../audioData/VAD_Kaggle/Data/Annotation/Noizeu...
717,../audioData/VAD_Kaggle/Data/Audio/Noizeus/Tra...,../audioData/VAD_Kaggle/Data/Annotation/Noizeu...


In [30]:
dataset_csv.to_csv('VADKaggle.csv', index=False)

In [51]:
#text_grid_fn = '../audioData/VAD_Kaggle/Data/Annotation/Male/TMIT/SI1060.TextGrid'
text_grid_fn = dataset_csv['annotation_files'][5]
grid = textgrids.TextGrid(text_grid_fn)

In [52]:
grid['silences']

[<Interval text="0" xmin=0.0 xmax=3.049>,
 <Interval text="1" xmin=3.049 xmax=3.833>,
 <Interval text="0" xmin=3.833 xmax=4.089>,
 <Interval text="1" xmin=4.089 xmax=4.680999999999999>,
 <Interval text="0" xmin=4.680999999999999 xmax=4.785>,
 <Interval text="1" xmin=4.785 xmax=5.369>,
 <Interval text="0" xmin=5.369 xmax=5.505000000000001>,
 <Interval text="1" xmin=5.505000000000001 xmax=6.665000000000001>,
 <Interval text="0" xmin=6.665000000000001 xmax=6.824999999999999>,
 <Interval text="1" xmin=6.824999999999999 xmax=7.945>,
 <Interval text="0" xmin=7.945 xmax=9.93>]

In [62]:
for i in grid['silences']:
    if i.text == '1':
        print(np.round(i.xmin, decimals=0), np.round(i.xmax, decimals=0), i.text)
#np.round(grid['silences'][6].xmin)

3.0 4.0 1
4.0 5.0 1
5.0 5.0 1
6.0 7.0 1
7.0 8.0 1


In [15]:
grid.keys()
grid.tier_to_csv('silences', 'test.csv')

In [43]:
#sample_aud = '/work/dpandya/LibriVox_Kaggle/BGnoise/supermarkt.wav'
sample_aud = 'audio.wav'
#sample_aud = '/work/dpandya/LibriVox_Kaggle/achtgesichterambiwasse/achtgesichterambiwasse_0003.wav'
aud, sr = torchaudio.load(sample_aud)
aud.shape, sr

(torch.Size([1, 48350976]), 16000)

In [44]:
import torchaudio.transforms as T

resampler = T.Resample(sr, new_freq=8000, dtype=aud.dtype)
resampled_aud = resampler(aud)
sr = 8000
resampled_aud.shape, sr

(torch.Size([1, 24175488]), 8000)

In [47]:
resampled_aud_np = resampled_aud.squeeze().numpy()
mfcc_feat = mfcc(resampled_aud_np[0:100*sr], sr)